In [1]:
!python --version
# use python 3.8.17

Python 3.8.17


In [2]:
!pip install experta

Defaulting to user installation because normal site-packages is not writeable


In [3]:
from experta import *
from rules import *
import numpy as np

In [4]:
# Initialize the knowledge engines.
suspicion_engine = SuspicionInvestigation()
tumor_engine = TumorStage()
nodes_engine = NodesStage()
metastasis_engine = MetastasisStage()
cancer_engine = CancerStage()


In [5]:
from utilities import Utils
from logistics import Costs
import re

c = Costs()
u = Utils()

# we consider suspicion, tumor, nodes, metastasis and cancer stages ready to be determined 
# if no relevant requirement is missing (None)
def ready_to_determine(my_dict):
    for value in my_dict.values():
        if value is None:
            return False
    return True

def determine(engine, fact, data):
    engine.reset()
    engine.declare(fact(**data))
    engine.run()
    return engine.get_result()

def exam_for_tumor(subject):
    print(subject.tumor_requirements)
    for attr_name, attr_value in subject.tumor_requirements.items():
        if attr_name == 'mass' and attr_value is None:
            subject.tumor_requirements['mass'] = random.choice([True, False])
            extra_cost = u.gaussian(c.pricelist["x_rays"][0], c.pricelist["x_rays"][1])
            extra_delay = u.gaussian(c.delay["x_rays"][0], c.delay["x_rays"][1])
            print('extra cost for chest x_rays: ', round(extra_cost, 2))
            print('extra delay for chest x_rays: ', int(extra_delay))
            subject.total_cost += round(extra_cost, 2) 
            subject.total_delay += int(extra_delay) 
            return
        
        if attr_name == 'diameter' and attr_value is None:
            if subject.tumor_requirements['mass'] and subject.tumor_requirements['mass'] == True:
                subject.tumor_requirements['diameter'] = round(random.uniform(0, 14), 2)
                rand = round(random.uniform(0, 10), 2)
                if rand <= 2:
                    subject.tumor_requirements['nearby_organs'] = True
                    subject.tumor_requirements['fna_and_pet_scan'] = True
                else:
                    subject.tumor_requirements['nearby_organs'] = False
                    subject.tumor_requirements['fna_and_pet_scan'] = True                    
            else:
                subject.tumor_requirements['diameter'] = 0.0
                subject.tumor_requirements['nearby_organs'] = False
                subject.tumor_requirements['fna_and_pet_scan'] = False
            extra_cost = u.gaussian(c.pricelist["ct_scan"][0], c.pricelist["ct_scan"][1])
            extra_delay = u.gaussian(c.delay["ct_scan"][0], c.delay["ct_scan"][1])
            print('extra cost for ct_scan: ', round(extra_cost, 2))
            print('extra delay for ct_scan: ', int(extra_delay))
            subject.total_cost += round(extra_cost, 2) 
            subject.total_delay += int(extra_delay)
            return
                
        if attr_name == 'bronchoscopesis' and attr_value is None:
            subject.tumor_requirements['bronchoscopesis'] = random.choice([True, False])
            extra_cost = u.gaussian(c.pricelist["bronchoscopesis"][0], c.pricelist["bronchoscopesis"][1])
            extra_delay = u.gaussian(c.delay["bronchoscopesis"][0], c.delay["bronchoscopesis"][1])
            print('extra cost for bronchoscopesis: ', round(extra_cost, 2))
            print('extra delay for bronchoscopesis: ', int(extra_delay))
            subject.total_cost += round(extra_cost, 2)
            subject.total_delay += int(extra_delay)
            return
        
        if attr_name == 'cytologic' and attr_value is None:
            subject.tumor_requirements['cytologic'] = random.choice([True, False])
            extra_cost = u.gaussian(c.pricelist["cytologic"][0], c.pricelist["cytologic"][1])
            extra_delay = u.gaussian(c.delay["cytologic"][0], c.delay["cytologic"][1])
            print('extra cost for cytologic: ', round(extra_cost, 2))
            print('extra delay for cytologic: ', int(extra_delay))
            subject.total_cost += round(extra_cost, 2) 
            subject.total_delay += int(extra_delay)
            return
        
        if attr_name == 'fna_and_pet_scan' and attr_value is None:
            subject.tumor_requirements['fna_and_pet_scan'] = random.choice([True, False])
            extra_cost = u.gaussian(c.pricelist["fna_and_pet_scan"][0], c.pricelist["fna_and_pet_scan"][1])
            extra_delay = u.gaussian(c.delay["fna_and_pet_scan"][0], c.delay["fna_and_pet_scan"][1])
            print('extra cost for fna_and_pet_scan: ', round(extra_cost, 2))
            print('extra delay for fna_and_pet_scan: ', int(extra_delay))
            subject.total_cost += round(extra_cost, 2) 
            subject.total_delay += int(extra_delay)
            return
            
def exam_for_nodes(subject):
    for attr_name, attr_value in subject.nodes_requirements.items():
        if attr_name == 'lymph_nodes_size' and attr_value is None:
            if subject.tumor_stage == 'T0':
                subject.nodes_requirements['lymph_nodes_size'] = 0.5
                subject.nodes_requirements['peribronchial_metastasis'] = False
                subject.nodes_requirements['mediastinal_metastasis'] = False
            elif re.match('T1.', subject.tumor_stage):
                subject.nodes_requirements['lymph_nodes_size'] = round(random.uniform(0.5, 1.5), 2)
                subject.nodes_requirements['peribronchial_metastasis'] = False
                subject.nodes_requirements['mediastinal_metastasis'] = False
            else:
                subject.nodes_requirements['lymph_nodes_size'] = round(random.uniform(1.6, 11), 2)
                subject.nodes_requirements['peribronchial_metastasis'] = random.choice([True, False])
                subject.nodes_requirements['mediastinal_metastasis'] = True

            extra_cost = u.gaussian(c.pricelist["ct_scan"][0], c.pricelist["ct_scan"][1])
            extra_delay = u.gaussian(c.delay["ct_scan"][0], c.delay["ct_scan"][1])
            print('extra cost for ct_scan: ', round(extra_cost, 2))
            print('extra delay for ct_scan: ', int(extra_delay))
            subject.total_cost += round(extra_cost, 2) 
            subject.total_delay += int(extra_delay) 
            return
        
        if attr_name == 'peribronchial_metastasis' and attr_value is None:
            subject.nodes_requirements['peribronchial_metastasis'] = random.choice([True, False])
            subject.nodes_requirements['mediastinal_metastasis'] = random.choice([True, False])
            extra_cost = u.gaussian(c.pricelist["pet_scan"][0], c.pricelist["pet_scan"][1])
            extra_delay = u.gaussian(c.delay["pet_scan"][0], c.delay["pet_scan"][1])
            print('extra cost for pet_scan: ', round(extra_cost, 2))
            print('extra delay for pet_scan: ', int(extra_delay))
            subject.total_cost += round(extra_cost, 2) 
            subject.total_delay += int(extra_delay) 
            return
        
        if attr_name == 'fna_positive' and attr_value is None:
            subject.nodes_requirements['fna_positive'] = random.choice([True, False])
            extra_cost = u.gaussian(c.pricelist["fna"][0], c.pricelist["fna"][1])
            extra_delay = u.gaussian(c.delay["fna"][0], c.delay["fna"][1])
            print('extra cost for fna: ', round(extra_cost, 2))
            print('extra delay for fna: ', int(extra_delay))
            subject.total_cost += round(extra_cost, 2) 
            subject.total_delay += int(extra_delay) 
            return        
        
def exam_for_metastasis(subject):
    for attr_name, attr_value in subject.metastasis_requirements.items():
        if attr_name == 'separate_tumor_nodules' and attr_value is None:
            subject.metastasis_requirements['separate_tumor_nodules'] = random.choice([True, False])
            extra_cost = u.gaussian(c.pricelist["scintigraphy"][0], c.pricelist["scintigraphy"][1])
            extra_delay = u.gaussian(c.delay["scintigraphy"][0], c.delay["scintigraphy"][1])
            print('extra cost for scintigraphy: ', round(extra_cost, 2))
            print('extra delay for scintigraphy: ', int(extra_delay))
            subject.total_cost += extra_cost 
            subject.total_delay += int(extra_delay) 
            return
        
        if attr_name == 'distant_metastasis' and attr_value is None:
            subject.metastasis_requirements['distant_metastasis'] = random.choice([True, False])
            extra_cost = u.gaussian(c.pricelist["mri_brain"][0], c.pricelist["mri_brain"][1])
            extra_delay = u.gaussian(c.delay["mri_brain"][0], c.delay["mri_brain"][1])
            print('extra cost for mri_brain: ', round(extra_cost, 2))
            print('extra delay for mri_brain: ', int(extra_delay))
            subject.total_cost += round(extra_cost, 2) 
            subject.total_delay += int(extra_delay) 
            return        

def estimate(subject):

    if subject.cancer_suspected == None:
        print('Suspected for cancer None')
        if ready_to_determine(subject.suspicion_requirements) == True:
            subject.cancer_suspected = determine(suspicion_engine, SUSPICION_FACTS, subject.suspicion_requirements)
        else:
            subject.set_suspicion_requirements()
            subject.cancer_suspected = determine(suspicion_engine, SUSPICION_FACTS, subject.suspicion_requirements)
        estimate(subject)
        
    elif subject.cancer_suspected == 'False':
        print('Suspected for cancer False')
        return
    
    else:
        print('Suspected for cancer True')
        
        if subject.tumor_stage == None:
            if ready_to_determine(subject.tumor_requirements) == True:
                subject.tumor_stage = determine(tumor_engine, TUMOR_FACTS, subject.tumor_requirements)
                print('Tumor estimated as: ', subject.tumor_stage)
                print('subject.tumor_requirements: ', subject.tumor_requirements)
            else:
                exam_for_tumor(subject)
                estimate(subject)        

        if subject.nodes_stage == None:
            if ready_to_determine(subject.nodes_requirements) == True:
                subject.nodes_stage = determine(nodes_engine, NODES_FACTS, subject.nodes_requirements)
                print('Nodes estimated as: ', subject.nodes_stage)
                print('subject.nodes_requirements: ', subject.nodes_requirements)
            else:
                exam_for_nodes(subject)
                estimate(subject)

        if subject.metastasis_stage == None:
            if ready_to_determine(subject.metastasis_requirements) == True:
                subject.metastasis_stage = determine(metastasis_engine, METASTASIS_FACTS, subject.metastasis_requirements)
                print('Metastasis estimated as: ', subject.metastasis_stage)
                print('subject.metastasis_requirements: ', subject.metastasis_requirements)
            else:
                exam_for_metastasis(subject)
                estimate(subject)

        if subject.cancer_stage == None:
            if subject.tumor_stage != None and subject.nodes_stage != None and subject.metastasis_stage != None:
#               if we have reached this point and either T,N or M have value None this means that no appropriate rule was found,
#               and no further exams are available for the respective T, N, M
#               therefore we should set the None to Tx, Nx or Mx respectively
                if subject.tumor_stage != None:
                    subject.tnm_requirements['T'] = subject.tumor_stage
                else:
                    subject.tnm_requirements['T'] = 'Tx'
                if subject.nodes_stage != None:
                    subject.tnm_requirements['N'] = subject.nodes_stage
                else:
                    subject.tnm_requirements['N'] = 'Nx'
                if subject.metastasis_stage != None:
                    subject.tnm_requirements['M'] = subject.metastasis_stage
                else:
                    subject.tnm_requirements['M'] = 'Mx'
                print(subject.tnm_requirements)
                subject.cancer_stage = determine(cancer_engine, TNM_FACTS, subject.tnm_requirements)
                print('Cancer estimated as: ', subject.cancer_stage)
                result = (f'Subject: {subject.id}, '
                        f'Cancer suspected: {subject.cancer_suspected}, '
                        f'Cancer diagnosis: {subject.cancer_stage}, '
                        f'Total cost: {subject.total_cost}, '
                        f'Total delay: {subject.total_delay}\n')
                print(result)
                with open('subjects_results.txt', 'a') as f:
                    # Write the data to the file
                    f.write(result)
                return
            else:
                estimate(subject)

In [6]:
import json
from synthesizer import *
synth = Synthesizer()

# Loading the data:
subjects = synth.load_subjects_from_json('subjects.json')
print('Total subjects: ', len(subjects))

# for each subject runfor key, value in subject.items():
for subject in subjects:
    print('Estimating for subject: ', subject.id)
    estimate(subject)

Total subjects:  100
Estimating for subject:  1
Suspected for cancer None
Suspected for cancer False
Estimating for subject:  2
Suspected for cancer None
Suspected for cancer False
Estimating for subject:  3
Suspected for cancer None
Suspected for cancer False
Estimating for subject:  4
Suspected for cancer None
Suspected for cancer False
Estimating for subject:  5
Suspected for cancer None
Suspected for cancer True
{'mass': None, 'diameter': None, 'bronchoscopesis': None, 'cytologic': None, 'nearby_organs': None, 'fna_and_pet_scan': None}
extra cost for chest x_rays:  8.25
extra delay for chest x_rays:  2
Suspected for cancer True
{'mass': True, 'diameter': None, 'bronchoscopesis': None, 'cytologic': None, 'nearby_organs': None, 'fna_and_pet_scan': None}
extra cost for ct_scan:  58.42
extra delay for ct_scan:  23
Suspected for cancer True
{'mass': True, 'diameter': 6.65, 'bronchoscopesis': None, 'cytologic': None, 'nearby_organs': False, 'fna_and_pet_scan': True}
extra cost for bronch